<p style="text-align: center;" ><font size="7" >Peprmint dataset creation setup</font></p>
<hr />

Author: Thibault Tubiana, Ph.D.  
version : 1.0

In [2]:
import os

# Dependancies


Installation with a environment.

1. install nb_conda_kernels to use your environment in conda (and only this, because mkdssp will not be loaded in the PATH without this solution)  
`conda install -c conda-forge nb_conda_kernels nodejs ipywidgets`

2. Create a environment for this peprmint datasate and use its kernel  
`conda create --name peprmint -c conda-forge -c salilab -c hydroid seaborn pandas biopandas biopython ipywidgets scipy numpy tqdm xmltodict pathlib dssp pytables termcolor requests bs4 lxml wordcloud nglview mdanalysis weasyprint openpyxl`  
Then install a last package with pypi  
`conda activate peprmint`  
`pip install pbxplore freesasa vg`
3. for progressbar usage (in base):  
`conda deactivate`  
`jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter nbextension enable --py widgetsnbextension`

4. For NGLVIEW  
`jupyter labextension install  nglview-js-widgets`

5. Don't forget to choose the peprmint environment for the kernel "`Python [conda env:peprmint]`"

6. LINUX AND MAC ONLY:  
if you want some speedup sometimes, install pandarallel `pip install pandarallel`

# folder definition

In [3]:
SETUP = {} #Dictionnary with ALL parameters

In [4]:
#Folder definition
from sys import platform
if platform == "linux" or platform == "linux2":
    PEPRMINT_FOLDER = "/mnt/d/work/projets/peprmint"
elif platform == "darwin":
    PEPRMINT_FOLDER = "/Users/thibault/Documents/WORK/peprmint"
else:
    raise ValueError("OS NOT FOUND")
WORKDIR = f"{PEPRMINT_FOLDER}/dataset/"
CATHFOLDER = f"{PEPRMINT_FOLDER}/databases/cath/"
ALFAFOLDFOLDER = f"{PEPRMINT_FOLDER}/databases/alfafold/"
PROSITEFOLDER = f"{PEPRMINT_FOLDER}/databases/prosite/"
UNIPROTFOLDER = f"{PEPRMINT_FOLDER}/databases/uniprot/"
FIGURESFOLDER = f"{PEPRMINT_FOLDER}/figures/"

SETUP["PEPRMINT_FOLDER"]=PEPRMINT_FOLDER
SETUP["WORKDIR"]=WORKDIR
SETUP["CATHFOLDER"]=CATHFOLDER
SETUP["PROSITEFOLDER"]=PROSITEFOLDER
SETUP["ALFAFOLDFOLDER"]=ALFAFOLDFOLDER
SETUP["UNIPROTFOLDER"]=UNIPROTFOLDER
SETUP["FIGURESFOLDER"]=FIGURESFOLDER

In [15]:
for k in SETUP:
    exec(f"{k}2 = SETUP['{k}']")

In [16]:
PROSITEFOLDER2

'/Users/thibault/Documents/WORK/peprmint/databases/prosite/'

# Create dictories


In [8]:
if not os.path.exists(PEPRMINT_FOLDER):
    os.makedirs(PEPRMINT_FOLDER)
if not os.path.exists(WORKDIR):
    os.makedirs(WORKDIR)
if not os.path.exists(FIGURESFOLDER):
    os.makedirs(FIGURESFOLDER)
if not os.path.exists(ALFAFOLDFOLDER):
    os.makedirs(ALFAFOLDFOLDER)
if not os.path.exists(UNIPROTFOLDER):
    os.makedirs(UNIPROTFOLDER)
if not os.path.exists(PROSITEFOLDER):
    os.makedirs(PROSITEFOLDER) #MSA will contains the alignments in "msa" format (FASTA). 
if not os.path.exists(CATHFOLDER):
    os.makedirs(CATHFOLDER)

# Cath and Prosite mapping

First we define the CATHVERSION because their is no 'latest' pointer. So if we want to update something, we need to update the version mannually

In [9]:
CATHVERSION = 'v4_2_0'

In [10]:
DOMAIN_PROSITE = {
    "PH": "PS50003",
    "C2": ["PS50004","PS51547"],
    "C1": "PS50081",  # Note : no C1 prosite on SMART but 2 C1 ProSite on Interprot (PS50081,PS00479), I took PS50081 since the data in PS00479 are in PS50081.
    "PX": "PS50195",
    # "FYVE":"PS50178",
    "FYVE": ["PS50178",'PS50089', 'PS00518','PS50016','PS01359','PS50014','PS00633','PS50119'],  # FYVE CAN BE THIS ONE TOO....
    # "PPASE_MYOTUBULARIN":"PS51339",# no GRAM domain found on prosite. Has to do this manually. Go on http://smart.embl-heidelberg.de/smart/do_annotation.pl?DOMAIN=GRAM&BLAST=DUMMY
    "BAR": "PS51021",  # 1URU is missing on prosite
    # "GLA":"PS50963",
    "ENTH": "PS50942",
    "SH2": "PS50001",
    "SEC14": "PS50191",
    "START": "PS50848",
    "C2DIS":"PS50022",
    "GLA": "PS50998",
    "PLD":"PS50035",
    "PLA":"PS00118",
    "ANNEXIN":"PS00223",
}
# Invert keys and values to have PROSITEID ==> DOMAIN
PROSITE_DOMAIN = {}
for key, value in DOMAIN_PROSITE.items():
    if type(value) == type([]):
        for subvalues in value:
            PROSITE_DOMAIN[subvalues] = key
    else:
        PROSITE_DOMAIN[value] = key
# PROSITE_DOMAIN = {v: k for k, v in DOMAIN_PROSITE.items()}

DOMAIN_CATH = {
    "PH": "2.30.29.30",
    "C2": "2.60.40.150",
    "C1": "3.30.60.20",
    "PX": "3.30.1520.10",
    "FYVE": "3.30.40.10",
    "BAR": "1.20.1270.60",
    "ENTH": "1.25.40.90",
    "SH2": "3.30.505.10",
    "SEC14": "3.40.525.10",
    "START": "3.30.530.20",
    "C2DIS": "2.60.120.260",
    "GLA":"2.40.20.10",
    "PLD":"3.20.20.190",
    "PLA":"1.20.90.10",
    "ANNEXIN":"1.10.220.10",
}

DOMAIN_INTERPRO = {
    "PH": "SSF50729",
    "C2": "SSF49562",
    "C1": None,
    "PX": "SSF64268",
    "FYVE": "SSF57903", #badly classified it looks like...
    "BAR": "SSF103657",
    "ENTH": "SSF48464",
    "SH2": "SSF55550",
    "SEC14": ["SSF52087","SSF46938"], #the CRAL TRIO domain is truncated in SSF.
    "START": "SSF55961",
    "C2DIS": "SSF49785",
    "GLA":None,
    "PLD":"SSF51695",
    "PLA":"G3DSA:1.20.90.10",
    "ANNEXIN":"SSF47874",
}

DOMAIN_INTERPRO_REFINE = {
    "PH": True,
    "C2": False,
    "C1": False,
    "PX": True,
    "FYVE": False,
    "BAR": False,
    "ENTH": False,
    "SH2": False,
    "SEC14": False,
    "START": True,
    "C2DIS": False,
    "GLA":False,
    "PLD":False,
    "PLA":True,
    "ANNEXIN":False,
}

# Invert keys and values to have CATHID ==> DOMAIN
CATH_DOMAIN = {v: k for k, v in DOMAIN_CATH.items()}
SUPERFAMILY = CATH_DOMAIN
SETUP["DOMAIN_PROSITE"] = DOMAIN_PROSITE
SETUP["PROSITE_DOMAIN"] = PROSITE_DOMAIN
SETUP["DOMAIN_CATH"] = DOMAIN_CATH
SETUP["CATH_DOMAIN"] = CATH_DOMAIN
SETUP["SUPERFAMILY"] = SUPERFAMILY


# TOOL FUNCTION


In [2]:
def selectUniquePerCluster(df, cathCluster, Uniref, withAlignment = True):
    """
    Return a datasert with only 1 data per choosed clusters.
    """
    
    if cathCluster not in ["S35","S60","S95","S100"]:
        raise ValueError('CathCluster given not in ["S35","S60","S95","S100"]')
    
    if Uniref not in ["uniref50","uniref90","uniref100"]:
        raise ValueError('CathCluster given not in ["uniref50","uniref90","uniref100"]')
    
    if withAlignment:
        df = df[~df.alignment_position.isnull()]
    
    cathdf = df.query("data_type == 'cathpdb'")
    seqdf = df.query("data_type == 'prosite'")
    
    def selectUniqueCath(group):
        uniqueNames = group.cathpdb.unique()
        select = uniqueNames[0]
        
        #return group.query("cathpdb == @select")
        return select
    
    def selectUniqueUniref(group,exclusion):
        uniqueNames = group.uniprot_acc.unique()
        select = uniqueNames[0]
        #return group.query("uniprot_acc == @select")
        if select not in exclusion:
            return select
        

    dfReprCathNames = cathdf.groupby(["domain",cathCluster]).apply(selectUniqueCath).to_numpy()
    
    excludeUniref = df.query("cathpdb in @dfReprCathNames").uniprot_acc.unique() #Structures are prior to sequences.
    dfReprUnirefNames = seqdf.groupby(["domain",Uniref]).apply(selectUniqueUniref, exclusion=excludeUniref).to_numpy()
    dfReprCath = cathdf.query("cathpdb in @dfReprCathNames")
    dfReprUniref = seqdf.query("uniprot_acc in @dfReprUnirefNames")
    
    return (pd.concat([dfReprCath,dfReprUniref]))

NameError: name 'DOMAIN_SEQ' is not defined

# Dataset description


 - _**Data related**_
   - **data_type**: Origin of the data (`cath` for PDBS, `prosite` for sequences)
   - **domain**: domain of the se
   - **cathpdb**: CATH domain ID (name of the PDB)
   - **pdb**: 4 char PDB Code
   - **uniprot_acc**: Uniprot Accession number (like 'P14262')
   - **uniprot_id**: Uniprot ID (like 'PLC_BACCE')
   - **domain**: domain Name
   - **location**: Location of the protein in the cell
   - **CR:prositeID**: Cross Reference: prosite IDs list in the uniprot record.
   - **CR:prositeName**: Cross Reference: prosite Names list in the uniprot record.
   
 - _**Residue related**_
   - **aatype**: Amino acid property classification
   - **residue_name**: amino acid name
   - **residue_number**: amino acid number
   - **residue_index**: 0-based residue number.
 
 - _**Atom related**_
   - **atom_name**: Name of the atom
   - **atom_number**: Atom numbering
   - **convhull_vertex**: True if the atom is vertex of the convexhull, False if it's not.
   - **density**: density of the atoms (number of neighbours)
   - **neighbours**: list of neighbours
   - **lowdens_protrusion**: True if the atom is a low density protrusion, False if it's not
   
 - _**Structure Related**_
   - **chain_id**: Chain identifier
   - **bfactor**: Bfactor of each atoms (if present)
   - **x_coord/y_coord/z_coord**: X/Y/Z coordinate
   - **resolution**: resolution of the PDB
   - **occupancy**: atom's occupancy
   - **sasa_rel**: accessible surface aera
   - **sec_struc**: secondary structure of the current amino acid
   - **sec_struc_segment**: secondary structure segment

 - _**Sequence related**_
   - **alignment_position**: position of the amino acid in the multiple alignment
   - **entropy**: Shannon Entropy of the alignment position
   - **ali_range**: begin-end of of the alignment in the full protein sequence (unipro_id). The full protein sequence is not in the dataset
   
 - _**Cluster Related**_
   - **S100** : Cath Cluster number at 100% of idendity
   - **S95** : Cath Cluster number at 95% of idendity
   - **S60** : Cath Cluster number at 60% of idendity
   - **S35** : Cath Cluster number at 35% of idendity
   - **uniref50**: Representative uniprot_acc for cluster with 50% of idendity
   - **uniref90**: Representative uniprot_acc for cluster with 90% of idendity
   - **uniref100**: Representative uniprot_acc for cluster with 100% of idendity
